In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# change the path to where your notebook is located
%cd "gdrive/My Drive/Colab Notebooks/NLI course"

/content/gdrive/My Drive/Colab Notebooks/NLI course


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
# Run it only once
from datasets import load_dataset
dataset = load_dataset("multi_woz_v22")

Extracting data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Run it only once
dataset.save_to_disk("dataset.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8437 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict.load_from_disk("dataset.hf")

In [ ]:
dir(dataset['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_w

In [ ]:
dataset['train'].features

{'dialogue_id': Value(dtype='string', id=None),
 'services': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'turns': Sequence(feature={'turn_id': Value(dtype='string', id=None), 'speaker': ClassLabel(names=['USER', 'SYSTEM'], id=None), 'utterance': Value(dtype='string', id=None), 'frames': Sequence(feature={'service': Value(dtype='string', id=None), 'state': {'active_intent': Value(dtype='string', id=None), 'requested_slots': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'slots_values': Sequence(feature={'slots_values_name': Value(dtype='string', id=None), 'slots_values_list': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, length=-1, id=None)}, 'slots': Sequence(feature={'slot': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None), 'start': Value(dtype='int32', id=None), 'exclusive_end': Value(dtype='int32', id=None), 'copy_from': Value(dtype='string', id=None), 'copy_from_value': Sequence(featur

In [ ]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
dataset['train'][0].keys()

dict_keys(['dialogue_id', 'services', 'turns'])

In [ ]:
print(len(dataset["train"]), len(dataset["validation"]), len(dataset["test"]))

8437 1000 1000


In [ ]:
set([service for dial in dataset['train'] for service in dial['services']])

{'attraction', 'bus', 'hospital', 'hotel', 'restaurant', 'taxi', 'train'}

In [ ]:
set([service for dial in dataset['train'] for turn_id,utt in enumerate(dial['turns']['utterance']) for service in dial['turns']['frames'][turn_id]['service']])

{'attraction',
 'bus',
 'hospital',
 'hotel',
 'police',
 'restaurant',
 'taxi',
 'train'}

In [ ]:
dataset['train'][112]

{'dialogue_id': 'SSNG0297.json',
 'services': ['hotel'],
 'turns': {'turn_id': ['0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13'],
  'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  'utterance': ['Can you find me a 3 star hotel in the centre?',
   'The Gonville Hotel meets your requirements. Would you like me to book you a room?',
   'Does it have free wifi? I need wifi.',
   'Yes, they do offer free internet. Would you like me to make a booking?',
   'Yes. I want to book it for 1 person. I want to stay for 5 nights starting from Wednesday.',
   'I am sorry Booking was unsuccessful,can I help you find another place to stay?',
   'Yes please. Another 3-star hotel in the centre with free wifi.',
   'I do not have another 3 star hotel in the centre that meets your requirements. Would you like to try another area?',
   'Do you have any hotels in the same price range as the Gonville Hotel?',
   'The only other hotel that

In [ ]:
def Dialogue_Act_Prediction(utterance, other_features_from_dialogue_history):
    return ['Hotel-Inform']

def Extract_and_Categorize_Spans(utterance, dialogue_acts, other_features_from_dialogue_history2):
    return [('hotel-bookpeople', '2'), ('hotel-bookstay', '2'), ('hotel-bookday', 'sunday'), ('restaurant-phone', '?')]

def Agent_Move_Prediction(dialogue_acts, extracted_information, other_features_from_dialogue_history3):
    return {"dialogue_acts": ["Booking-Request", "Hotel-Recommend", "Restaurant-Inform"],
            "retrieved_information": {"to_be_provided": set(['hotel-area', 'hotel-name', 'restaurant-name', 'restaurant-phone', 'restaurant-address', 'booking-ref']),
                                      "to_be_requested": set(['booking-bookday', 'booking-bookpeople', 'restaurant-pricerange'])}}

In [ ]:
from collections import Counter


def print_output(*output):
    if do_print_dialogue_details:
        if len(output) == 1:
            print(output[0])
        elif len(output) == 2:
            print(output[0], output[1])
        elif len(output) == 3:
            print(output[0], output[1], output[2])
        elif len(output) == 4:
            print(output[0], output[1], output[2], output[3])
        else:
            print(output)

def count_matches(ground_truth_list, predicted_list):
    no_gt = len(ground_truth_list)
    no_predicted = len(predicted_list)
    no_correct = no_gt - sum((Counter(ground_truth_list) - Counter(predicted_list)).values())
    return no_gt, no_predicted, no_correct

def get_metrics(no_gt_global, no_predicted_global, no_correct_global):
    precision = 1.0*no_correct_global/no_predicted_global if no_predicted_global else 0.0
    recall = 1.0*no_correct_global/no_gt_global if no_gt_global else 0.0
    f1_score = 2.0*precision*recall/(precision+recall) if precision and recall else 0.0
    return precision, recall, f1_score

In [ ]:
import pandas as pd

global do_print_dialogue_details
do_print_dialogue_details = False
print_ground_truth_structures = False
print_predicted = False

no_user_moves_gt_global = 0 # "gt" == "ground truth"
no_user_moves_predicted_global = 0
no_user_moves_correct_global = 0

no_user_info_gt_global = 0
no_user_info_predicted_global = 0
no_user_info_correct_global = 0

no_moves_gt_global = 0
no_moves_predicted_global = 0
no_moves_correct_global = 0

no_agent_info_to_provide_gt_global = 0
no_agent_info_to_provide_predicted_global = 0
no_agent_info_to_provide_correct_global = 0

no_agent_info_to_request_gt_global = 0
no_agent_info_to_request_predicted_global = 0
no_agent_info_to_request_correct_global = 0

training_examples_user_da = []

n_dialogues_to_evaluate = 10
n_evaluated = 0
for d,dial in enumerate(dataset['validation']):
    if n_evaluated>=n_dialogues_to_evaluate:
        break
    # skip dialogues that are not in the hotel or restaurant domain
    if not any(set(dial['turns']['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(dial['turns']['utterance'])):
        continue
    print_output("Dialogue ID:", dial['dialogue_id'])
    compulsory_slots_hotel  = set(['hotel-bookpeople', 'hotel-bookstay', 'hotel-name', 'hotel-bookday']) # as an example, to be adjusted
    compulsory_slots_restaurant  = set(['restaurant-name']) # as an example, to be adjusted
    filled_slots = set()
    speaker_str = {0: 'User', 1: 'Agent'}
    turns = dial['turns']
    prev_user_turn = ""
    prev_agent_turn = ""
    prev_user_acts = []
    prev_agent_acts = []
    for turn_id,utt in enumerate(turns['utterance']):
        speaker = speaker_str[turns['speaker'][turn_id]]
        if speaker == "User":
            if turn_id>1:
                prev_user_turn = turns['utterance'][turn_id-2]
            if turn_id>0:
                prev_agent_turn = turns['utterance'][turn_id-1]

            print_output("User's turn")
            print_output("User's utterance: "+utt)

            print_output("Extraction")

            indent = " "*4
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
            dialogue_acts_ground_truth = dialogue_acts.copy()
            print_output(indent, "Dialogue acts:", dialogue_acts)

            if dialogue_acts_ground_truth:
                training_examples_user_da.append({"input": " | ".join([prev_user_turn, ", ".join(prev_user_acts),
                                                                       prev_agent_turn, ", ".join(prev_agent_acts),
                                                                       utt]),
                                                  "output": ", ".join(dialogue_acts_ground_truth)})
            dialogue_acts_predicted = Dialogue_Act_Prediction(utt, other_features_from_dialogue_history={})
            if print_ground_truth_structures:
                print(dialogue_acts_ground_truth)

            # evaluate user's dialogue acts
            no_gt, no_predicted, no_correct = count_matches(dialogue_acts_ground_truth, dialogue_acts_predicted)
            no_user_moves_gt_global += no_gt
            no_user_moves_predicted_global += no_predicted
            no_user_moves_correct_global += no_correct
            if print_predicted:
                print(indent, "User's dialogue acts predicted:", dialogue_acts_predicted, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output(indent, "Extracted information:")
            print_output(indent, "Spans")
            extracted_information_not_mapped_ground_truth = []
            extracted_information_ground_truth = []
            extracted_information_per_dialogue_act_ground_truth = {}
            span_info = turns['dialogue_acts'][turn_id]['span_info']
            for span_i in range(len(span_info['span_start'])):
                act_type = span_info['act_type'][span_i]
                span_name = span_info['act_slot_name'][span_i]
                span_value = span_info['act_slot_value'][span_i]
                span_range = [span_info['span_start'][span_i], span_info['span_end'][span_i]]
                span_value_as_in_utterance = utt[span_info['span_start'][span_i]: span_info['span_end'][span_i]]
                print_output(indent*2, span_value + ("" if span_value_as_in_utterance==span_value else " ("+span_value_as_in_utterance+")"), span_range)
                if not act_type in extracted_information_per_dialogue_act_ground_truth:
                    extracted_information_per_dialogue_act_ground_truth[act_type] = []
                act_category = act_type.split("-")[0].lower()
                extracted_information_not_mapped_ground_truth.append(tuple([act_category+"-"+span_name, span_value_as_in_utterance]))
                if act_category in ["hotel", "restaurant", "general"]:
                    extracted_information_ground_truth.append(tuple([act_category+"-"+span_name, span_value]))
                extracted_information_per_dialogue_act_ground_truth[act_type].append(tuple([span_name, span_value]))

            print_output(indent, "Categorized information")
            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            for act_i in range(len(slot_names_per_act)):
                slot_names_values_per_act = [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i]))]
                print_output(indent*2, dialogue_acts[act_i], slot_names_values_per_act)
                if dialogue_acts[act_i].startswith("Hotel") or dialogue_acts[act_i].startswith("Restaurant"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none" and slot_values_per_act[act_i][slot_i] == "?":
                            if not dialogue_acts[act_i] in extracted_information_per_dialogue_act_ground_truth:
                                extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]] = []
                            extracted_information_not_mapped_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            extracted_information_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))

            extracted_information = Extract_and_Categorize_Spans(utt, dialogue_acts_ground_truth, other_features_from_dialogue_history2={})

            if print_ground_truth_structures:
                print(extracted_information_not_mapped_ground_truth)
                print(extracted_information_ground_truth)
                print(extracted_information_per_dialogue_act_ground_truth)

            # evaluate information extraction only if there are Hotel or Restaurant or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(extracted_information_ground_truth, extracted_information)
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") for da in dialogue_acts_ground_truth):
                no_user_info_gt_global += no_gt
                no_user_info_predicted_global += no_predicted
                no_user_info_correct_global += no_correct
                if print_predicted:
                    print(indent, "Extracted and categorized information (predicted):", extracted_information, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output("Reasoning")
            services = turns['frames'][turn_id]['service']
            print_output(indent, "Services:", services)

            not_empty_intents = [intent for intent in turns['frames'][turn_id]['state'] if intent['requested_slots'] or intent['slots_values']['slots_values_name']]
            if not_empty_intents:
                print_output(indent, "Intents")
                for intent in not_empty_intents:
                    print_output(indent*2, "Active intent:", intent['active_intent'])
                    requested_slots = intent['requested_slots']
                    if requested_slots:
                        print_output(indent*2, "Requested slots:", requested_slots)
                    if intent['slots_values']['slots_values_name']:
                        slot_names = intent['slots_values']['slots_values_name']
                        slot_values = intent['slots_values']['slots_values_list']
                        filled_slots.update(slot_names)
                        print_output(indent*2, "Filled slots:")
                        for slot_i in range(len(slot_names)):
                            print_output(indent*3, slot_names[slot_i]+": ", slot_values[slot_i])
                    print_output(indent*2, "--------------")

            print_output(indent, "Missing slots (Hotel):", compulsory_slots_hotel - filled_slots)
            print_output(indent, "Missing slots (Restaurant):", compulsory_slots_restaurant - filled_slots)

            prev_user_acts = dialogue_acts_ground_truth
        elif speaker == "Agent":
            if turn_id>1:
                prev_agent_turn = turns['utterance'][turn_id-2]
            if turn_id>0:
                prev_user_turn = turns['utterance'][turn_id-1]

            print_output("Agent's turn")

            print_output("Planning")

            indent = " "*4
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
            print_output(indent, "Agent's move (dialogue acts):", dialogue_acts)

            print_output("Retrieval")

            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]

            print_output(indent, "Retrieved information:")
            for act_i in range(len(slot_names_per_act)):
                print_output(indent*2, dialogue_acts[act_i])
                print_output(indent*2, "To be provided:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]!="?"])
                print_output(indent*2, "To be requested:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]=="?"])
                print_output(indent*2, "--------------")

            agent_move_ground_truth = {
                "dialogue_acts": dialogue_acts,
                "retrieved_information": {"to_be_provided": set(),  # only unique names of the slots
                                          "to_be_requested": set()  # only unique names of the slots
                                          },
                "retrieved_information_per_dialogue_act": {} # non-none names of the slots and values grouped per dialogue act
                }

            for act_i in range(len(slot_names_per_act)):
                da = dialogue_acts[act_i]
                if da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") or da.startswith("general"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none":
                            if not dialogue_acts[act_i] in agent_move_ground_truth["retrieved_information_per_dialogue_act"]:
                                agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]] = []
                            agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            if slot_values_per_act[act_i][slot_i]!="?":
                                agent_move_ground_truth["retrieved_information"]["to_be_provided"].add(dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i])
                            else:
                                agent_move_ground_truth["retrieved_information"]["to_be_requested"].add(dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i])

            agent_move_predicted = Agent_Move_Prediction(dialogue_acts, extracted_information_per_dialogue_act_ground_truth, other_features_from_dialogue_history3={})
            if print_ground_truth_structures:
                print(agent_move_ground_truth)

            if print_predicted:
                print("Planning predicted")

            # evaluate agent's dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["dialogue_acts"], agent_move_predicted["dialogue_acts"])
            no_moves_gt_global += no_gt
            no_moves_predicted_global += no_predicted
            no_moves_correct_global += no_correct
            if print_predicted:
                print(indent, "Agent's dialogue acts predicted:", agent_move_predicted["dialogue_acts"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            # evaluate "retrieved_information" -> "to_be_provided" only if there are Hotel or Restaurant or Booking or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["retrieved_information"]["to_be_provided"], agent_move_predicted["retrieved_information"]["to_be_provided"])
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in agent_move_ground_truth["dialogue_acts"]):
                no_agent_info_to_provide_gt_global += no_gt
                no_agent_info_to_provide_predicted_global += no_predicted
                no_agent_info_to_provide_correct_global += no_correct
                if print_predicted:
                    print(indent, "Info to be provided predicted:", agent_move_predicted["retrieved_information"]["to_be_provided"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            # evaluate "retrieved_information" -> "to_be_requested" only if there are Hotel or Restaurant or Booking or general dialogue acts
            no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["retrieved_information"]["to_be_requested"], agent_move_predicted["retrieved_information"]["to_be_requested"])
            if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in agent_move_ground_truth["dialogue_acts"]):
                no_agent_info_to_request_gt_global += no_gt
                no_agent_info_to_request_predicted_global += no_predicted
                no_agent_info_to_request_correct_global += no_correct
                if print_predicted:
                    print(indent, "Info to be requested predicted:", agent_move_predicted["retrieved_information"]["to_be_requested"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output("Agent's utterance: "+utt)
            prev_agent_acts = agent_move_ground_truth["dialogue_acts"]


        print_output("-------------------------------------------------------------------")
        print_output("-------------------------------------------------------------------")
    n_evaluated+=1

print("Dialogue acts in the user's move prediction")
precision, recall, f1_score = get_metrics(no_user_moves_gt_global, no_user_moves_predicted_global, no_user_moves_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Extracted information from user's utterance")
precision, recall, f1_score = get_metrics(no_user_info_gt_global, no_user_info_predicted_global, no_user_info_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Dialogue acts in the agent's move prediction")
precision, recall, f1_score = get_metrics(no_moves_gt_global, no_moves_predicted_global, no_moves_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Info to be provided by the agent")
precision, recall, f1_score = get_metrics(no_agent_info_to_provide_gt_global, no_agent_info_to_provide_predicted_global, no_agent_info_to_provide_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Info to be requested by the agent")
precision, recall, f1_score = get_metrics(no_agent_info_to_request_gt_global, no_agent_info_to_request_predicted_global, no_agent_info_to_request_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))

Dialogue acts in the user's move prediction
Precision: 0.115789, Recall: 0.102804, F1-score: 0.108911
Extracted information from user's utterance
Precision: 0.018868, Recall: 0.068966, F1-score: 0.029630
Dialogue acts in the agent's move prediction
Precision: 0.059649, Recall: 0.117241, F1-score: 0.079070
Info to be provided by the agent
Precision: 0.056497, Recall: 0.357143, F1-score: 0.097561
Info to be requested by the agent
Precision: 0.028249, Recall: 0.357143, F1-score: 0.052356


In [ ]:
training_examples_user_da

[{'input': ' |  |  |  | i need a place to dine in the center thats expensive',
  'output': 'Restaurant-Inform'},
 {'input': 'i need a place to dine in the center thats expensive | Restaurant-Inform | I have several options for you; do you prefer African, Asian, or British food? | Restaurant-Inform, Restaurant-Select | Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?',
  'output': 'Restaurant-Request'},
 {'input': 'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation? | Restaurant-Request | There is an Afrian place named Bedouin in the centre. How does that sound? | Restaurant-Inform | Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel?',
  'output': 'Hotel-Inform, Restaurant-Request'},
 {'input': "Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel? | Hotel-Inform, Restau

In [ ]:
df = pd.DataFrame.from_dict(training_examples_user_da)
df

,input,output
0,| | | | i need a place to dine in the cent...,Restaurant-Inform
1,i need a place to dine in the center thats exp...,Restaurant-Request
2,"Any sort of food would be fine, as long as it ...","Hotel-Inform, Restaurant-Request"
3,"Sounds good, could I get that phone number? Al...",Hotel-Inform
4,Yes. Can you book it for me? | Hotel-Inform | ...,Hotel-Inform
...,...,...
64,"| | | | Yeah, could you recommend a good g...",Restaurant-Inform
65,"Yeah, could you recommend a good gastropub? | ...",Restaurant-Inform
66,I would like to book a table at the Backstreet...,Attraction-Inform
67,"Yes, any suggestions of museums found in the e...",Attraction-Request


In [ ]:
print(df["input"].tolist()[-4])
print(df["output"].tolist()[-4])

Yeah, could you recommend a good gastropub? | Restaurant-Inform | Backstreet Bistro. It's expensive though. There is a moderately priced one called The Cow Pizza Kitchen and Bar if preferred. | Restaurant-Inform, Restaurant-Recommend | I would like to book a table at the Backstreet Bistro for 5 people at 16:00 on Thursday.
Restaurant-Inform


In [ ]:
max([len(input.split()) for input in df["input"].tolist()])

71

In [ ]:
df = pd.DataFrame.from_dict(training_examples_user_da)
df["one label"] = df["output"].apply(lambda x: not "," in x)
df = df[df["one label"]==True][["input", "output"]]
max([len(input.split()) for input in df["input"].tolist()])

98

In [ ]:
# entire training set
df = pd.DataFrame.from_dict(training_examples_user_da)
df["one label"] = df["output"].apply(lambda x: not "," in x)
df = df[df["one label"]==True][["input", "output"]]
max([len(input.split()) for input in df["input"].tolist()])

95

In [ ]:
df.to_csv("user_da_train.csv", sep = "\t", index=False)
#df.to_csv("user_da_validation.csv", sep = "\t", index=False)
df.to_csv("user_da_test.csv", sep = "\t", index=False)

In [ ]:
import pandas as pd
#pd.read_csv("user_da_train.csv", sep = "\t").to_csv("user_da_train.csv", index=False)
pd.read_csv("user_da_validation.csv", sep = "\t").to_csv("user_da_validation.csv", index=False)
pd.read_csv("user_da_test.csv", sep = "\t").to_csv("user_da_test.csv", index=False)

https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification

In [ ]:
#! python run_classification.py --max_seq_length 128 --label_column_name "output" --train_file "user_da_train.csv" --validation_file "user_da_validation.csv" --test_file "user_da_test.csv" --text_column_names "input" --text_column_delimiter " | " --shuffle_train_dataset

In [ ]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9nxnman9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9nxnman9
  Resolved https://github.com/huggingface/transformers to commit 2ac5b9325ed3b54950c6c61fd5838ac6e55a9fe1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8012744 sha256=74918a1c2364a4935975572a558be2e97e1c932587762578c077d474472f9427
  Stored in directory: /tmp/pip-ephem-wheel-cache-0qb2s0ws/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successful

In [ ]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
! python run_classification.py --model_name_or_path  bert-large-uncased \
    --train_file "user_da_train.csv" \
    --validation_file "user_da_validation.csv" \
    --test_file "user_da_test.csv" \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --text_column_name "input" \
    --text_column_delimiter "\n" \
    --label_column_name "output" \
    --do_train \
    --do_eval \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir results_bert

2023-11-13 18:44:02.122513: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 18:44:02.122569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 18:44:02.122609: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 18:44:03.783321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Colab Notebooks/NLI course/run_classification.py", line 51, in <module>
    check_min_version("4.36.0.dev0")
  File "/usr/local/lib/python3.10/dist-packages/transformers/

In [ ]:
! python run_classification.py --model_name_or_path  bert-large-uncased \
    --train_file "user_da_train.csv" \
    --validation_file "user_da_validation.csv" \
    --test_file "user_da_test.csv" \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --text_column_name "input" \
    --text_column_delimiter "\n" \
    --label_column_name "output" \
    --do_train \
    --do_eval \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir results_bert

2023-11-13 18:46:16.901721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 18:46:16.910148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 18:46:16.910212: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 18:46:20.699172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Colab Notebooks/NLI course/run_classification.py", line 763, in <module>
    main()
  File "/content/gdrive/MyDrive/Colab Notebooks/NLI course/run_classification.py", line

In [ ]:
! python run_classification.py --model_name_or_path  bert-large-uncased \
    --train_file "user_da_train.csv" \
    --validation_file "user_da_validation.csv" \
    --test_file "user_da_test.csv" \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --text_column_name "input" \
    --text_column_delimiter "\n" \
    --label_column_name "output" \
    --do_train \
    --do_eval \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir results_bert_large_uncased_user_da

2023-11-13 18:47:12.486862: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 18:47:12.486919: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 18:47:12.486959: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 18:47:14.192406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/13/2023 18:47:16 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2023 18:47:16 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
! python run_classification.py --model_name_or_path  bert-large-uncased \
    --train_file "user_da_train.csv" \
    --validation_file "user_da_validation.csv" \
    --test_file "user_da_test.csv" \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --text_column_name "input" \
    --text_column_delimiter "\n" \
    --label_column_name "output" \
    --do_train \
    --do_eval \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir results_bert_large_uncased_user_da

2023-11-13 18:50:10.311772: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 18:50:10.311832: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 18:50:10.311886: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 18:50:11.460441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/13/2023 18:50:13 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2023 18:50:13 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
# --test_file shall be provided only along with --do_predict
! python run_classification.py --model_name_or_path  bert-large-uncased --train_file "user_da_train.csv" --validation_file "user_da_validation.csv" --test_file "user_da_test.csv" --shuffle_train_dataset --metric_name accuracy --text_column_name "input" --label_column_name "output" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 1 --output_dir results_bert_large_uncased_user_da

2023-11-13 22:22:12.779947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 22:22:12.780012: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 22:22:12.780067: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 22:22:16.510872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/13/2023 22:22:23 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/13/2023 22:22:23 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

df_test = pd.read_csv("user_da_test.csv")
df_test_preds = pd.read_csv("results_bert_large_uncased_user_da/predict_results.txt", sep="\t")
true_labels = df_test["output"].tolist()
pred_labels = df_test_preds["prediction"].tolist()
print(classification_report(true_labels, pred_labels))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

 Attraction-Inform       0.95      0.94      0.94       303
Attraction-Request       0.90      0.90      0.90       199
      Hotel-Inform       0.97      0.98      0.98      1172
     Hotel-Request       0.86      0.85      0.85       169
     Police-Inform       0.00      0.00      0.00         1
 Restaurant-Inform       0.98      0.97      0.98      1132
Restaurant-Request       0.85      0.84      0.85       167
       Taxi-Inform       0.92      0.95      0.94       166
      Taxi-Request       0.69      0.87      0.77        23
      Train-Inform       0.99      0.99      0.99       777
     Train-Request       0.95      0.95      0.95       117
       general-bye       0.97      1.00      0.99       225
     general-greet       1.00      0.50      0.67         6
     general-thank       0.98      0.97      0.98       693

          accuracy                           0.96      5150
         macro avg       0.86      0.8

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
! python run_classification.py --model_name_or_path  bert-base-uncased \
    --dataset_name ${dataset} \
    --dataset_config_name ${subset} \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --text_column_name "review_title,review_body,product_category" \
    --text_column_delimiter "\n" \
    --label_column_name stars \
    --do_train \
    --do_eval \
    --max_seq_length 512 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir /tmp/${dataset}_${subset}/

Running your trained model for a single inference

https://huggingface.co/FredZhang7/one-for-all-toxicity-v3

Applied to our model:

In [ ]:
text = "I am also looking for The Riverside Brasserie. | Restaurant-Inform | riverside brasserie is a modern european restaurant in the centre. Its address is Doubletree by Hilton Cambridge Granta Place Mill Lane and postcode is cb21rt. Want to book it? | Booking-Inform, Restaurant-Inform | Yes, I would like to book it and get a reference number."
gt_output = 'Restaurant-Request'

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = AutoModelForSequenceClassification.from_pretrained("results_bert_large_uncased_user_da").to(device)
model_config = json.load(open("results_bert_large_uncased_user_da/config.json", "r"))

encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

print('device:', device)
input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

print(predicted_labels)
pred_id = str(predicted_labels.cpu().detach().numpy()[0])
print(model_config["id2label"][pred_id])

device: cpu
tensor([9])
Restaurant-Request
